# 1. Data Preprocessing

## 1.1 Import Dependencies

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/test_round2.csv
./data/train_round2.csv
./data/graph_round2.csv
./data/test.csv
./data/graph.csv
./data/submission_round2.csv
./data/submission.csv
./data/train_round2-1205.csv
./data/train.csv
./data/covid-full.csv


## 1.2 Load Data

In [3]:
data_dir = 'data'
temporal_data_path = os.path.join(data_dir, 'train.csv')
mobility_data_path = os.path.join(data_dir, 'graph_round2.csv')

temporal_data = pd.read_csv(temporal_data_path)
temporal_data.describe()

# train.csv 04/12/2020 to 08/31/2020
# train_round2.csv 04/12/2020 to 11/22/2020
# train_round2-1205.csv 04/12/2020 to 12/05/2020
# to predict 12/07/2020 to 12/13/2020, forecast 8 days, take the last 7 days

,ID,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
count,7100.000000,7100.000000,7100.000000,5771.000000,7100.000000,7100.000000,7.100000e+03,5047.000000,7100.000000,7100.000000,5047.000000
mean,3549.500000,56374.339577,2323.883099,20422.274649,37635.120986,747.273790,6.427069e+05,6250.044185,3.653231,9666.099116,12.197144
std,2049.737788,95058.665054,4742.274729,39262.213465,79067.101832,637.397470,1.127555e+06,13880.694435,2.074042,7779.560336,5.255183
min,0.000000,270.000000,0.000000,0.000000,-989.000000,32.771004,5.459000e+03,10.000000,0.000000,460.300152,2.302896
25%,1774.750000,6097.500000,153.000000,1614.500000,2849.000000,224.815702,1.000775e+05,595.000000,1.928925,3379.288445,8.387037
50%,3549.500000,22892.000000,681.000000,5881.000000,10409.000000,540.836745,2.643235e+05,2068.000000,3.362027,7842.820591,11.351161
75%,5324.250000,62959.250000,2334.000000,26103.000000,32106.500000,1126.282666,7.048395e+05,6134.500000,4.797564,14127.521137,15.396088
max,7099.000000,712475.000000,32957.000000,507499.000000,699453.000000,3187.773001,1.137330e+07,89995.000000,9.741481,49502.860687,38.501190


In [4]:
states = list(np.unique(temporal_data['Province_State']))
state_cum_temporal_data = dict.fromkeys(states, None)
state_temporal_data = dict.fromkeys(states, None)
# state_cum_temporal_scaler = defaultdict(StandardScaler)
# state_temporal_scaler = defaultdict(StandardScaler)
dropped_attr = ['Date',
                'Active',
                'ID', 
                'Province_State', 
                'Incident_Rate', 
                'Recovered', 
                'People_Tested', 
                'People_Hospitalized', 
                'Mortality_Rate', 
                'Testing_Rate', 
                'Hospitalization_Rate']

for s in states:
    df_filter = temporal_data['Province_State'] == s
    state_df = temporal_data[df_filter]
    # Daily difference data
    state_temporal_data[s] = state_df.drop(dropped_attr, 1)
    for col in state_temporal_data[s]:
        data = state_temporal_data[s][col].tolist()
        diff = [i - j for i, j in zip(data, [*[data[0]], *data[:len(data) - 1]])]
        state_temporal_data[s][col] = diff
    save_columns = state_temporal_data[s].columns
#     state_temporal_data[s] = state_temporal_scaler[s].fit_transform(state_temporal_data[s])
    state_temporal_data[s] = pd.DataFrame(state_temporal_data[s], columns=save_columns)
    # Daily cumulative data
    state_cum_temporal_data[s] = state_df.drop(dropped_attr, 1)
    save_columns = state_cum_temporal_data[s].columns
#     state_cum_temporal_data[s] = state_cum_temporal_scaler[s].fit_transform(state_cum_temporal_data[s])
    state_cum_temporal_data[s] = pd.DataFrame(state_cum_temporal_data[s], columns=save_columns)

In [5]:
state_temporal_data['South Dakota'].values

array([[  0,   0],
       [138,   0],
       [120,   0],
       [180,   0],
       [143,   1],
       [100,   0],
       [131,   0],
       [ 93,   0],
       [ 50,   0],
       [ 70,   1],
       [103,   1],
       [ 98,   0],
       [ 84,   1],
       [107,   0],
       [ 65,   1],
       [ 32,   0],
       [ 69,   0],
       [ 60,   2],
       [ 76,   4],
       [ 76,   4],
       [ 63,   0],
       [ 43,   0],
       [ 37,   0],
       [ 53,   3],
       [ 59,   5],
       [126,   2],
       [239,   0],
       [248,   3],
       [124,   0],
       [ 97,   0],
       [ 49,   5],
       [ 69,   0],
       [ 60,   4],
       [ 95,   1],
       [ 72,   0],
       [ 28,   0],
       [ 40,   0],
       [ 58,   2],
       [ 92,   0],
       [  0,   2],
       [179,   2],
       [108,   0],
       [ 99,   0],
       [ 23,   0],
       [ 67,   0],
       [ 57,   4],
       [ 83,   0],
       [ 73,   5],
       [ 94,   3],
       [ 33,   0],
       [ 41,   0],
       [ 33,   0],
       [ 95,

## Train a RNN Model with LSTM

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# window_size is use n days to predict (n+1)th day
window_size = 14
forecast_days = 26

# test model for current_state
current_state = 'South Dakota'
data_training = state_temporal_data[current_state]

In [7]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
data_training = scaler.fit_transform(data_training)

In [8]:
X_train = []
y_train = []

data_training = np.array(data_training)
data_training_np = np.array(data_training)

for i in range(data_training.shape[0] - window_size):
    X_train.append(data_training[i : i + window_size])
    y_train.append(data_training_np[i + window_size])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [9]:
X_train.shape

(128, 14, 2)

In [10]:
y_train.shape

(128, 2)

In [11]:
model = keras.Sequential()

model.add(LSTM(units=20, activation = 'relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))

model.add(LSTM(units=10, activation = 'relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=20, activation = 'relu', return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(units = 2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 14, 20)            1840      
_________________________________________________________________
dropout (Dropout)            (None, 14, 20)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 14, 10)            1240      
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 10)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                2480      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4

In [12]:
model.compile(optimizer='adam', loss = 'mean_squared_logarithmic_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

In [13]:
prediction_queue = X_train[-1:]
prediction_queue = np.array(prediction_queue)
prediction_queue

array([[[0.13804173, 0.        ],
        [0.13001605, 0.16666667],
        [0.20064205, 0.16666667],
        [0.20064205, 0.33333333],
        [0.30979133, 0.33333333],
        [0.40288925, 0.16666667],
        [0.22632424, 0.16666667],
        [0.23916533, 0.        ],
        [0.12841091, 0.        ],
        [0.105939  , 0.16666667],
        [1.        , 0.        ],
        [0.51845907, 0.5       ],
        [0.68218299, 0.33333333],
        [0.60995185, 0.        ]]])

In [14]:
y_case_forecast = []
y_death_forecast = []

for i in range(forecast_days):  
    y_pred = model.predict(prediction_queue)
    y_case_forecast.append(y_pred[0][0])
    y_death_forecast.append(y_pred[0][1])
    prediction_queue = np.append(prediction_queue, y_pred)
    prediction_queue = np.delete(prediction_queue, 0)
    prediction_queue = np.delete(prediction_queue, 0)
    prediction_queue = prediction_queue.reshape(1,int(prediction_queue.shape[0] / 2),2)


In [15]:
y_case_forecast

[0.15054926,
 0.1540772,
 0.15694618,
 0.15834467,
 0.15900567,
 0.15813576,
 0.155826,
 0.15424323,
 0.15194625,
 0.15026122,
 0.14885923,
 0.14206243,
 0.13969533,
 0.13637719,
 0.13361397,
 0.13350627,
 0.13336,
 0.13318932,
 0.13300699,
 0.13282341,
 0.1326513,
 0.13249819,
 0.13236085,
 0.13224217,
 0.13213934,
 0.13205256]

In [16]:
y_death_forecast

[0.17282563,
 0.17298742,
 0.17336757,
 0.1737726,
 0.17417556,
 0.1744139,
 0.1744021,
 0.17446722,
 0.17438388,
 0.17429574,
 0.1742582,
 0.17353524,
 0.17315573,
 0.17262375,
 0.1720882,
 0.17208037,
 0.17206374,
 0.17203887,
 0.17200974,
 0.17197776,
 0.17194313,
 0.17190927,
 0.17187718,
 0.17184882,
 0.1718237,
 0.17180018]

In [17]:
case_scale = 1/scaler.scale_[0]
death_scale = 1/scaler.scale_[1]

In [18]:
y_case_forecast = [case_scale * i for i in y_case_forecast]
y_death_forecast = [death_scale * i for i in y_death_forecast]

In [19]:
last_day_case = state_cum_temporal_data[current_state]['Confirmed'].iloc[-1]
last_day_death = state_cum_temporal_data[current_state]['Deaths'].iloc[-1]

y_cum_case_forecast = []
y_cum_death_forecast = []

cum_case = last_day_case
cum_death = last_day_death
for case in y_case_forecast:
    cum_case += case
    y_cum_case_forecast.append(cum_case)
    
for death in y_death_forecast:
    cum_death += death
    y_cum_death_forecast.append(cum_death)

In [20]:
y_cum_case_forecast

[13602.79219070077,
 13698.782287597656,
 13796.559759140015,
 13895.208489254117,
 13994.269022747874,
 14092.787599265575,
 14189.867198735476,
 14285.96073153615,
 14380.623243182898,
 14474.235985308886,
 14566.975287258625,
 14655.480178415775,
 14742.51036991179,
 14827.47335653007,
 14910.714862242341,
 14993.889267370105,
 15076.97254641354,
 15159.949493125081,
 15242.812847867608,
 15325.561830833554,
 15408.203590258956,
 15490.749962508678,
 15573.210769444704,
 15655.597643196583,
 15737.920452043414,
 15820.189194232225]

In [21]:
y_cum_death_forecast

[168.03695380687714,
 169.07487830519676,
 170.11508375406265,
 171.157719373703,
 172.20277273654938,
 173.24925616383553,
 174.29566878080368,
 175.34247210621834,
 176.38877537846565,
 177.4345498085022,
 178.48009902238846,
 179.52131047844887,
 180.5602448284626,
 181.5959873497486,
 182.62851658463478,
 183.66099879145622,
 184.6933812201023,
 185.7256144285202,
 186.7576728463173,
 187.78953939676285,
 188.82119816541672,
 189.8526538014412,
 190.8839168548584,
 191.9150097668171,
 192.94595193862915,
 193.97675302624702]

# 2. Train a RNN with LSTM for Every State

## 2.1 Import Dependencies

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# window_size is use n-1 days to predict nth day
window_size = 14
forecast_days = 26

## 2.2 Helper Functions

In [7]:
states = state_temporal_data.keys()

def get_training_data(state):
    state_temporal_train = state_temporal_data[state]
    scaler = MinMaxScaler()
    state_temporal_train = scaler.fit_transform(state_temporal_train)
    
    X_train = []
    y_train = []

    state_temporal_train_np = np.array(state_temporal_train)

    for i in range(state_temporal_train.shape[0] - window_size):
        X_train.append(state_temporal_train[i : i + window_size])
        y_train.append(state_temporal_train_np[i + window_size])

    return np.array(X_train), np.array(y_train), scaler
    

def get_model(X_train):
    model = keras.Sequential()

    model.add(LSTM(units=20, activation = 'relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.3))

    model.add(LSTM(units=10, activation = 'relu', return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(units=20, activation = 'relu', return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(units = 2))
    
    return model


def get_prediction_queue(X_train):
    prediction_queue = X_train[-1:]
    return np.array(prediction_queue)


def predict(model, queue, scaler):
    prediction_queue = queue
    
    for i in range(forecast_days):  
        y_pred = model.predict(prediction_queue)
        prediction_queue = np.append(prediction_queue, y_pred)
        prediction_queue = np.delete(prediction_queue, 0)
        prediction_queue = np.delete(prediction_queue, 0)
        prediction_queue = prediction_queue.reshape(1, prediction_queue.shape[0] // 2, 2)
        
    y_case_forecast = []
    y_death_forecast = []

    for i in range(forecast_days):
        y_case_forecast.append(prediction_queue[0][i + window_size - forecast_days][0])
        y_death_forecast.append(prediction_queue[0][i + window_size - forecast_days][1])
        
    case_scale = 1 / scaler.scale_[0]
    death_scale = 1 / scaler.scale_[1]
    
    return [case_scale * i for i in y_case_forecast], [death_scale * i for i in y_death_forecast]


def get_cum_forecast(state, y_case_forecast, y_death_forecast):
    last_day_case = state_cum_temporal_data[state]['Confirmed'].iloc[-1]
    last_day_death = state_cum_temporal_data[state]['Deaths'].iloc[-1]

    y_cum_case_forecast = []
    y_cum_death_forecast = []

    cum_case = last_day_case
    cum_death = last_day_death
    for case in y_case_forecast:
        cum_case += case
        y_cum_case_forecast.append(cum_case)

    for death in y_death_forecast:
        cum_death += death
        y_cum_death_forecast.append(cum_death)
        
    return y_cum_case_forecast, y_cum_death_forecast

## 2.3 Train and Predict

In [8]:
forecasted = {}

for state in states:
    X_train, y_train, scaler = get_training_data(state)

    model = get_model(X_train)
    model.compile(optimizer='adam', loss = 'mean_squared_logarithmic_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    
    y_case_forecast, y_death_forecast = predict(model, get_prediction_queue(X_train), scaler)
    y_cum_case_forecast, y_cum_death_forecast = get_cum_forecast(state, y_case_forecast, y_death_forecast)
    print(state)
    print(y_cum_case_forecast)
    print(y_cum_death_forecast)
    
    forecasted[state] = (y_cum_case_forecast, y_cum_death_forecast)

Alabama
[126979.78747370839, 127904.44114139676, 128824.55818790197, 129741.63973844051, 130655.98593226075, 131562.65076339245, 132460.06604358554, 133346.68110105395, 134221.91074290872, 135090.66722911596, 135949.9861305058, 136806.84514838457, 137757.88390153646, 138737.4294486642, 139659.21692237258, 140583.87059006095, 141503.98763656616, 142421.0691871047, 143335.41538092494, 144242.08021205664, 145139.49549224973, 146026.11054971814, 146901.3401915729, 147770.09667778015, 148629.41557917, 149486.27459704876]
[2197.2977681159973, 2212.6329308748245, 2227.9230493307114, 2243.1858211755753, 2258.4252153635025, 2273.585151016712, 2288.6470915675163, 2303.5922399163246, 2318.4127985835075, 2333.1636936068535, 2347.8106160759926, 2362.43275475502, 2378.0337228775024, 2393.9805276989937, 2409.278295814991, 2424.613458573818, 2439.903577029705, 2455.166348874569, 2470.405743062496, 2485.565678715706, 2500.62761926651, 2515.5727676153183, 2530.393326282501, 2545.144221305847, 2559.79114

Georgia
[271715.8578019738, 272935.4255180061, 274136.1806191057, 275322.2050085962, 276497.7092542052, 277667.1493470669, 278831.53932620585, 279991.4787648767, 281150.18484693766, 282312.13902387023, 283474.15459251404, 284638.01083670557, 285867.1179353446, 287129.4485642165, 288374.3063661903, 289593.8740822226, 290794.6291833222, 291980.6535728127, 293156.1578184217, 294325.5979112834, 295489.98789042234, 296649.9273290932, 297808.63341115415, 298970.5875880867, 300132.60315673053, 301296.45940092206]
[5665.409798622131, 5697.441763520241, 5729.132115364075, 5760.518790960312, 5791.637941896915, 5822.546026170254, 5853.270523786545, 5883.826981782913, 5914.251817166805, 5944.601577013731, 5974.852964401245, 6005.041409194469, 6038.040362715721, 6070.443704009056, 6102.853502631187, 6134.885467529297, 6166.575819373131, 6197.962494969368, 6229.0816459059715, 6259.98973017931, 6290.714227795601, 6321.270685791969, 6351.695521175861, 6382.045281022787, 6412.296668410301, 6442.4851132

Maine
[4550.832282662392, 4572.656485080719, 4594.467761933804, 4616.257675111294, 4638.021346330643, 4659.75638589263, 4681.46678212285, 4703.155508607626, 4724.8219484984875, 4746.464613825083, 4768.080104857683, 4789.671298652887, 4812.1315754354, 4833.948319166899, 4855.78060182929, 4877.604804247618, 4899.416081100702, 4921.2059942781925, 4942.969665497541, 4964.704705059528, 4986.415101289749, 5008.103827774525, 5029.770267665386, 5051.4129329919815, 5073.028424024582, 5094.619617819786]
[133.31473383307457, 134.6290171444416, 135.94257071614265, 137.2548925280571, 138.56569147109985, 139.8748208284378, 141.18251359462738, 142.4889467060566, 143.79407867789268, 145.09781777858734, 146.39995408058167, 147.7006535232067, 149.0503377020359, 150.36384335160255, 151.67857718467712, 152.99286049604416, 154.3064140677452, 155.61873587965965, 156.9295348227024, 158.23866418004036, 159.54635694622993, 160.85279005765915, 162.15792202949524, 163.4616611301899, 164.76379743218422, 166.06449

Nevada
[69756.07646143436, 70285.81501448154, 70819.78170001507, 71358.40661716461, 71899.49230492115, 72441.33122420311, 72987.03041040897, 73538.05605781078, 74095.80623412132, 74660.56023144722, 75229.14163780212, 75798.9066426754, 76338.59631633759, 76874.07223284245, 77402.14869427681, 77931.88724732399, 78465.85393285751, 79004.47885000706, 79545.5645377636, 80087.40345704556, 80633.10264325142, 81184.12829065323, 81741.87846696377, 82306.63246428967, 82875.21387064457, 83444.97887551785]
[1319.1216400265694, 1333.2545372843742, 1347.4377085268497, 1361.6790501773357, 1375.9473587572575, 1390.2185600996017, 1404.53963175416, 1418.9331230521202, 1433.4201295673847, 1448.0048958957195, 1462.6408307254314, 1477.2886879742146, 1491.5946252644062, 1505.8024486899376, 1519.924088716507, 1534.0569859743118, 1548.2401572167873, 1562.4814988672733, 1576.749807447195, 1591.0210087895393, 1605.3420804440975, 1619.7355717420578, 1634.2225782573223, 1648.8073445856571, 1663.443279415369, 1678

Oregon
[26856.231201916933, 26997.43662622571, 27136.689187705517, 27273.850102692842, 27408.716696947813, 27541.906865239143, 27673.78474292159, 27804.67683851719, 27934.622250407934, 28063.55798459053, 28191.20136716962, 28317.35084462166, 28469.30556604266, 28619.098259985447, 28762.32946190238, 28903.534886211157, 29042.787447690964, 29179.94836267829, 29314.81495693326, 29448.00512522459, 29579.883002907038, 29710.775098502636, 29840.72051039338, 29969.656244575977, 30097.299627155066, 30223.449104607105]
[461.26568937301636, 463.5084618330002, 465.7291293144226, 467.92574125528336, 470.0956549048424, 472.2465744614601, 474.3831704258919, 476.50965893268585, 478.6266499757767, 480.7334088087082, 482.82634848356247, 484.9027941226959, 487.277151286602, 489.62258088588715, 491.8882702589035, 494.13104271888733, 496.35171020030975, 498.5483221411705, 500.7182357907295, 502.86915534734726, 505.005751311779, 507.132239818573, 509.2492308616638, 511.35598969459534, 513.4489293694496, 51

Vermont
[1630.4870748519897, 1636.9741353690624, 1643.4611787945032, 1649.9482029229403, 1656.4352077543736, 1662.922195494175, 1669.4091677963734, 1675.8961285203695, 1682.3830809742212, 1688.8700290173292, 1695.3569737523794, 1701.8439184874296, 1708.3155237585306, 1714.800205230713, 1721.2872800827026, 1727.7743405997753, 1734.261384025216, 1740.7484081536531, 1747.2354129850864, 1753.7224007248878, 1760.2093730270863, 1766.6963337510824, 1773.1832862049341, 1779.670234248042, 1786.1571789830923, 1792.6441237181425]
[59.00035697221756, 60.00071394443512, 61.0010712146759, 62.00142952799797, 63.00178948044777, 64.00215151906013, 65.0025160908699, 66.0028834939003, 67.00325387716293, 68.00362768769264, 69.00400492548943, 70.00438544154167, 71.00356566905975, 72.00398403406143, 73.00434100627899, 74.00469797849655, 75.00505524873734, 76.0054135620594, 77.0057735145092, 78.00613555312157, 79.00650012493134, 80.00686752796173, 81.00723791122437, 82.00761172175407, 83.00798895955086, 84.0

## 2.4 Generate Submission File

In [9]:
forecastID = [x for x in range(forecast_days * 50)]
deaths = []
confirmed = []

for i in range(forecast_days):
    for s in states:
        print(f"Day {i+1}/26, {s}")
        confirmed.append(forecasted[s][0][i])
        deaths.append(forecasted[s][1][i])
        
final = pd.DataFrame(list(zip(forecastID, confirmed, deaths)), 
               columns =['ForecastID', 'Confirmed', 'Deaths']) 

final.to_csv("submission.csv", index=False)

Day 1/26, Alabama
Day 1/26, Alaska
Day 1/26, Arizona
Day 1/26, Arkansas
Day 1/26, California
Day 1/26, Colorado
Day 1/26, Connecticut
Day 1/26, Delaware
Day 1/26, Florida
Day 1/26, Georgia
Day 1/26, Hawaii
Day 1/26, Idaho
Day 1/26, Illinois
Day 1/26, Indiana
Day 1/26, Iowa
Day 1/26, Kansas
Day 1/26, Kentucky
Day 1/26, Louisiana
Day 1/26, Maine
Day 1/26, Maryland
Day 1/26, Massachusetts
Day 1/26, Michigan
Day 1/26, Minnesota
Day 1/26, Mississippi
Day 1/26, Missouri
Day 1/26, Montana
Day 1/26, Nebraska
Day 1/26, Nevada
Day 1/26, New Hampshire
Day 1/26, New Jersey
Day 1/26, New Mexico
Day 1/26, New York
Day 1/26, North Carolina
Day 1/26, North Dakota
Day 1/26, Ohio
Day 1/26, Oklahoma
Day 1/26, Oregon
Day 1/26, Pennsylvania
Day 1/26, Rhode Island
Day 1/26, South Carolina
Day 1/26, South Dakota
Day 1/26, Tennessee
Day 1/26, Texas
Day 1/26, Utah
Day 1/26, Vermont
Day 1/26, Virginia
Day 1/26, Washington
Day 1/26, West Virginia
Day 1/26, Wisconsin
Day 1/26, Wyoming
Day 2/26, Alabama
Day 2/26, 